In [38]:
from cracknuts.trace import ScarrTraceDataset
# 加载 scarr 格式的数据集
ds = ScarrTraceDataset.load(r'D:\work\ahcc\90.project\cracknuts\demo\jupyter\dataset\20250509174950.zarr')
# ds = ScarrTraceDataset.load(r'D:\project\cracknuts\demo\jupyter\dataset\20250313022104.zarr')

In [39]:
# Print dataset info
ds.info()

Channel: ['1']
Trace:   1000, 20480
Data:    1000 plaintext: 32 ciphertext: 16 key: 16 extended: None

In [ ]:
from zarr.hierarchy import Group

x: Group = ds.get_origin_data()["/0/0/"]

In [ ]:
# Show trace panel
import cracknuts as cn
pt = cn.panel_trace()
pt.set_trace_dataset(ds)
pt

In [ ]:
# Change trace index.
# show_trace 接收两个索引参数，第一个参数是通道索引，第二个是曲线索引，两个参数支持高级索引
# 需要注意的是通道A和B的名称在数据里为0和1，而其索引则根曲线文件中存储的通道数量有关系，如果只有一个通道时，无论哪个通道索引都为0，如果同时有A B两个通道则A通道索引为0，B通道索引为1，可以通过 ds.channel_names 进行确认当前曲线存储的时哪个通道

# 展示第一通道的前10条曲线
# tp.show_trace[1, :10]
# 展示第一通道的后10条曲线
# tp.show_trace[1, -10:]
# 展示第一通道的 10-20 曲线
# tp.show_trace[1, 10:20]
# 展示所有(两个)通道的前10条中的偶数位曲线
pt.show_trace[:, :10:2]

In [ ]:
# 获取第一通道的前10条曲线数据（np.ndarray）
ds.trace[0, :10]

In [ ]:
# 获取第一通道的前10条明文（当前版本该数据包含明文和密文）
ds.data[0, :10]

In [ ]:
# 加载 numpy 格式 dataset
from cracknuts.trace import NumpyTraceDataset

ds2 = NumpyTraceDataset.load(r'D:\project\cracknuts\demo\jupyter\dataset\20250313152352.npy')

In [ ]:
# 获取第二通道的前10条曲线数据（np.ndarray）
ds2.trace[1, :10]

In [ ]:
# 获取第二通道的前10条明文（当前版本该数据包含明文和密文）数据（np.ndarray）
ds2.data[1, :2]

*分析数据*

In [27]:
from scarr.engines.cpa import CPA as cpa
from scarr.file_handling.trace_handler import TraceHandler as th
from scarr.model_values.sbox_weight import SboxWeight
from scarr.container.container import Container, ContainerOptions
import numpy as np
from smartcard.util import toHexString

In [40]:
handler2 = th(fileName=r"D:\work\ahcc\90.project\cracknuts\demo\jupyter\dataset\20250509174950.zarr") 

opened zarr file  D:\work\ahcc\90.project\cracknuts\demo\jupyter\dataset\20250509174950.zarr


In [38]:
from cracknuts.trace import ScarrTraceDataset
# 加载 scarr 格式的数据集
ds = ScarrTraceDataset.load(r'D:\work\ahcc\90.project\cracknuts\demo\jupyter\dataset\20250509174950.zarr')
# ds = ScarrTraceDataset.load(r'D:\project\cracknuts\demo\jupyter\dataset\20250313022104.zarr')

In [29]:
import zarr

z = zarr.open(r'D:\work\ahcc\90.project\cracknuts\demo\jupyter\dataset\20250508221320.zarr')

z.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='0', nodes=(Node(disabled=True, name…

In [30]:
print(' '.join(f'{x:02x}' for x in z["/0/0/key"][0]))

11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff


In [31]:
print(' '.join(f'{x:02x}' for x in z["/0/0/plaintext"][0]))

7e 1f 38 5a e4 71 25 04 bc f7 8d 70 11 bb 8f 28


In [41]:
model = SboxWeight() 
engine2 = cpa(model)

In [42]:
container2 = Container(options=ContainerOptions(engine=engine2, handler=handler2), model_positions = [x for x in range(16)])
container2.run()

In [43]:
results2 = container2.engine.get_result()

In [44]:
results = np.squeeze(engine2.get_candidate())

In [45]:
toHexString(list(results))

'11 22 33 44 55 66 77 88 99 00 AA BB CC DD EE FF'

In [8]:
result = np.squeeze(results2)
result = result[0]

In [9]:
candidate = [None for _ in range(result.shape[0])]

for i in range(result.shape[0]):
    candidate[i] = np.unravel_index(np.abs(result[i, :]).argmax(), result[i, :].shape[0:])[0]

In [10]:
candidates = np.empty(256, dtype=np.float64)
for k in range(256):
    candidates[k] = result[k,candidate[k]]
ck = np.abs(candidates).argmax()

In [11]:
for _i in range(0, 10):
    print('第',str(_i+1),'个候选值：',hex(np.abs(candidates).argsort()[::-1][_i]),'， 对应的相关系数为：',str(candidates[np.abs(candidates).argsort()[::-1][_i]]))

第 1 个候选值： 0x84 ， 对应的相关系数为： 0.4498041570186615
第 2 个候选值： 0x30 ， 对应的相关系数为： -0.44673195481300354
第 3 个候选值： 0x56 ， 对应的相关系数为： -0.4383695125579834
第 4 个候选值： 0xd3 ， 对应的相关系数为： 0.431272029876709
第 5 个候选值： 0x6e ， 对应的相关系数为： 0.4298516511917114
第 6 个候选值： 0xe5 ， 对应的相关系数为： 0.4277685582637787
第 7 个候选值： 0xc4 ， 对应的相关系数为： -0.42412635684013367
第 8 个候选值： 0x72 ， 对应的相关系数为： -0.42084071040153503
第 9 个候选值： 0xe ， 对应的相关系数为： -0.41936182975769043
第 10 个候选值： 0xb6 ， 对应的相关系数为： -0.4181382954120636


In [12]:
backgroud_numpy = np.delete(results2[0, 0, 0, :, :3000], 123, axis=0)

In [13]:
heighlight_numpy = results2[0, 0, 0, 0x11, :3000][np.newaxis, :]

In [14]:
results2[0, 0, 0, :, :3000].shape

(256, 3000)

In [15]:
np.save('./rs.npy', results2[0, 0, 0, :, :3000])

In [16]:
result_numpy = np.vstack([backgroud_numpy, heighlight_numpy])

In [17]:
result_numpy.shape

(256, 3000)

In [18]:
pt2 = cn.panel_trace()
pt2.set_numpy_data(result_numpy)
pt2

NameError: name 'cn' is not defined

In [19]:
pt2.show_all_trace()

NameError: name 'pt2' is not defined

In [ ]:
pt2.highlight([0x11])

In [ ]:
pt2.change_percent_range(0, 100)

In [ ]:
pt2.change_range(1000, 1500)